# EDA Advanced Lesson 

As usual, we import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np

## Covariance and Correlation

Covariance and correlation are two mathematical concepts which are commonly used in statistics. They are used to determine the relationship between two variables. The covariance is used to measure the linear relationship between two variables. On the other hand, the correlation is used to measure both the strength and direction of the _linear relationship_ between two variables.

Covariance is a measure of how much two random variables vary together. It’s similar to variance, but where variance tells you how a single variable varies, covariance tells you how two variables vary together.

Correlation (coefficient) is a _normalized_ measure of covariance that is easier to understand, as it provides quantitative measurements of the statistical dependence between two random variables. The correlation coefficient is a value that indicates the strength of the relationship between variables. The coefficient can take any values from -1 to 1. The interpretations of the values are:

- **-1**: Perfect negative linear correlation
- **-0.8**: Strong negative linear correlation
- **-0.5**: Moderate negative linear correlation
- **-0.2**: Weak negative linear correlation
- **0**: No linear correlation
- **0.2**: Weak positive linear correlation
- **0.5**: Moderate positive linear correlation
- **0.8**: Strong positive linear correlation
- **1**: Perfect positive linear correlation


Here, we'll use DataFrames of stock prices and volumes obtained from Yahoo! Finance available in binary Python pickle files.

In [ ]:
price = pd.read_pickle("../data/yahoo_price.pkl")
volume = pd.read_pickle("../data/yahoo_volume.pkl")

In [ ]:
price

In [ ]:
volume

Compute percent changes of the prices using a window function (we will explain window functions in the later section).

In [ ]:
returns = price.pct_change()

returns.tail()

Compute the correlation and covariance between the returns of `MSFT` and `IBM`:

In [ ]:
returns["MSFT"].cov(returns["IBM"])

In [ ]:
returns["MSFT"].corr(returns["IBM"])

You can also get the full (pair-wise) correlation or covariance matrix as a DataFrame:

In [ ]:
returns.cov()

In [ ]:
returns.corr()

You can also compute pair-wise correlations between a DataFrame’s columns or rows with another Series or DataFrame. Passing a Series returns a Series with the correlation value computed for each column:

In [ ]:
returns.corrwith(returns["IBM"])

Passing a DataFrame computes the correlations of matching column names.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(price.corr(), annot=True, cmap="coolwarm")
plt.title("Stock Price Correlation")
plt.show()


In [ ]:
returns.corrwith(volume)

## Hierarchical Indexing

Hierarchical indexing (MultiIndex) allows you to have multiple (two or more) _index levels_ on an axis. It enables "higher dimensional" data in a lower dimensional data structure.

You create a hierarchical index by simply passing a list of arrays to the index argument of a pandas DataFrame or Series.

In [ ]:
data = pd.Series(np.random.uniform(size=9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

In [ ]:
data.index

You can use _partial indexing_ to select subsets of data:

In [ ]:
data["b"]

In [ ]:
data["b":"c"]

In [ ]:
data.loc[["b", "d"]]

You can also select from "inner" level:

In [ ]:
data.loc[:, 2]

Hierarchical indexing works on both axes.

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                        index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                        columns=[['Ohio', 'Ohio', 'Colorado'],
                        ['Green', 'Red', 'Green']])

frame

Setting names on the axes work as usual:

In [ ]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]

frame

In [ ]:
frame.index.nlevels

Partial indexing works on columns too:

In [22]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

You may need to rearrange the order of the levels on an axis. The `swaplevel` method will swap the levels in the MultiIndex on a particular axis. The default is to swap the levels on the rows:

In [23]:
frame.swaplevel()

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [24]:
frame.swaplevel(0, 1, axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

You can also sort by a single level or subset of levels:

In [25]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

> Swap the levels on the rows then sort the index by level `0`.

It's common to use one or more columns from a DataFrame as the row index.

In [26]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1), 
                      "c": ["one", "one", "one", "two", "two", "two", "two"], 
                      "d": [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


`set_index` will return a new DataFrame using one or more of its columns as the index.

In [27]:
frame2 = frame.set_index(["c", "d"])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

`reset_index` does the opposite of `set_index` and turns the index back into a column.

In [28]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


You can choose to drop the columns when resetting index:

In [29]:
frame2.reset_index(drop=True)

,a,b
0,0,7
1,1,6
2,2,5
3,3,4
4,4,3
5,5,2
6,6,1


## Date Time Data

Pandas is oriented towards working with arrays of dates, whether used as an axis index or a column.

The `to_datetime` method parses may different kinds of date representations:

In [33]:
dates = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]

pd.to_datetime(dates)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

It uses `NaT` (Not a Time) as null values for datetime data.

In [34]:
idx = pd.to_datetime(dates + [None])

idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [32]:
pd.isna(idx)

array([False, False,  True])

Standard Python uses the `datetime` module to handle date and time data. Pandas has a `Timestamp` object that is similar to the `datetime` object. Pandas also has a `Timedelta` object that is similar to the `timedelta` object.

If you use `datetime` objects as index to a Series or DataFrame, Pandas will automatically convert them to `DatetimeIndex` objects.

In [35]:
from datetime import datetime

In [36]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.standard_normal(6), index=dates)

ts

2011-01-02   -0.855604
2011-01-05    0.541300
2011-01-07   -1.808605
2011-01-08   -1.346026
2011-01-10   -0.858657
2011-01-12   -0.291040
dtype: float64

In [37]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

Like other Series, arithmetic operations between differently indexed time series automatically align on the dates:

In [38]:
# [::2] selects every second element
ts + ts[::2]

2011-01-02   -1.711207
2011-01-05         NaN
2011-01-07   -3.617210
2011-01-08         NaN
2011-01-10   -1.717313
2011-01-12         NaN
dtype: float64

`DatetimeIndex` is an array of `Timestamp` objects.

In [39]:
ts.index[0]

Timestamp('2011-01-02 00:00:00')

You can index by passing a `datetime`, `Timestamp` or `string` that is interpretable as a date:

In [40]:
ts[datetime(2011, 1, 7)]

-1.8086049153878296

In [41]:
ts[pd.Timestamp("2011-01-07")]

-1.8086049153878296

In [42]:
ts["2011-01-07"]

-1.8086049153878296

You can even specify the year or year-month strings:

In [43]:
# date_range generate an array of dates
longer_ts = pd.Series(np.random.standard_normal(1000), 
                      index=pd.date_range("2000-01-01", periods=1000))

longer_ts

2000-01-01    1.209563
2000-01-02   -1.752024
2000-01-03   -0.557996
2000-01-04   -0.478732
2000-01-05    0.793416
                ...   
2002-09-22   -0.084994
2002-09-23    0.018668
2002-09-24    0.952290
2002-09-25   -0.273381
2002-09-26    1.135529
Freq: D, Length: 1000, dtype: float64

In [44]:
longer_ts["2001"]

2001-01-01   -0.769450
2001-01-02    2.335972
2001-01-03    0.048691
2001-01-04   -0.177086
2001-01-05    0.513998
                ...   
2001-12-27    1.126381
2001-12-28    1.034228
2001-12-29    0.341097
2001-12-30   -1.419137
2001-12-31    0.671264
Freq: D, Length: 365, dtype: float64

In [45]:
longer_ts["2001-05"]

2001-05-01    0.310996
2001-05-02   -0.407801
2001-05-03   -0.466253
2001-05-04   -1.303087
2001-05-05   -1.901594
2001-05-06    0.397777
2001-05-07    0.583716
2001-05-08    0.545907
2001-05-09    1.322200
2001-05-10    0.424361
2001-05-11   -1.111969
2001-05-12    0.698554
2001-05-13   -0.468391
2001-05-14   -1.401969
2001-05-15    1.007210
2001-05-16   -2.310289
2001-05-17    1.425275
2001-05-18   -0.097826
2001-05-19    1.572814
2001-05-20    0.598128
2001-05-21   -0.804727
2001-05-22    0.006620
2001-05-23    1.797338
2001-05-24   -0.289783
2001-05-25   -1.601142
2001-05-26   -0.644424
2001-05-27   -1.166336
2001-05-28   -0.530988
2001-05-29   -0.046929
2001-05-30   -1.607252
2001-05-31   -2.112099
Freq: D, dtype: float64

Or slicing:

In [46]:
longer_ts["2001-05":]

2001-05-01    0.310996
2001-05-02   -0.407801
2001-05-03   -0.466253
2001-05-04   -1.303087
2001-05-05   -1.901594
                ...   
2002-09-22   -0.084994
2002-09-23    0.018668
2002-09-24    0.952290
2002-09-25   -0.273381
2002-09-26    1.135529
Freq: D, Length: 514, dtype: float64

> Use `date_range` to generate a Series of random values from 1-31st January 2023. Then slice the Series to return data from 5-15th January.

In [49]:
# 1️⃣ Create a date range for the whole month of January 2023
dates = pd.date_range(start="2023-01-01", end="2023-01-31")

# 2️⃣ Generate random values (one per day)
values = np.random.randn(len(dates))   # random numbers from a normal distribution

# 3️⃣ Create a time series (Series with datetime index)
ts = pd.Series(values, index=dates)

# 4️⃣ Slice the data from 5th to 15th January
subset = ts["2023-01-05":"2023-01-15"]

print(subset)


2023-01-05   -1.220844
2023-01-06    0.208864
2023-01-07   -1.959670
2023-01-08   -1.328186
2023-01-09    0.196861
2023-01-10    0.738467
2023-01-11    0.171368
2023-01-12   -0.115648
2023-01-13   -0.301104
2023-01-14   -1.478522
2023-01-15   -0.719844
Freq: D, dtype: float64


In [47]:
price = pd.read_pickle('../data/yahoo_price.pkl')

In [50]:
price

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...
2016-10-17,117.550003,779.960022,154.770004,57.220001
2016-10-18,117.470001,795.260010,150.720001,57.660000
2016-10-19,117.120003,801.500000,151.259995,57.529999


In [51]:
price.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2016-10-10', '2016-10-11', '2016-10-12', '2016-10-13',
               '2016-10-14', '2016-10-17', '2016-10-18', '2016-10-19',
               '2016-10-20', '2016-10-21'],
              dtype='datetime64[ns]', name='Date', length=1714, freq=None)

You can access other attributes like `day_of_week` or `month`:

In [ ]:
price.index.day_of_week
#5 trading days in a week: 0 (Monday) to 4 (Friday)

Int64Index([0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
            ...
            0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
           dtype='int64', name='Date', length=1714)

In [53]:
price.index.month

Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
            ...
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
           dtype='int64', name='Date', length=1714)

If the datetime is in a column instead of the index, you can use the `dt` accessor to access the datetime properties.

In [54]:
price_reindex = price.reset_index()

price_reindex

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...,...
1709,2016-10-17,117.550003,779.960022,154.770004,57.220001
1710,2016-10-18,117.470001,795.260010,150.720001,57.660000
1711,2016-10-19,117.120003,801.500000,151.259995,57.529999
1712,2016-10-20,117.059998,796.969971,151.520004,57.250000


In [55]:
price_reindex["Date"].dt.day_name()

0          Monday
1         Tuesday
2       Wednesday
3        Thursday
4          Friday
          ...    
1709       Monday
1710      Tuesday
1711    Wednesday
1712     Thursday
1713       Friday
Name: Date, Length: 1714, dtype: object

> Get the week of year from the date column and create a new column `week_of_year`.

In [58]:
df['week_of_year'] = df['date'].dt.isocalendar().week

df

,date,week_of_year
0,2023-01-01,52
1,2023-01-02,1
2,2023-01-03,1
3,2023-01-04,1
4,2023-01-05,1
5,2023-01-06,1
6,2023-01-07,1
7,2023-01-08,1
8,2023-01-09,2
9,2023-01-10,2


In [59]:
df['week_of_year'] = df['date'].dt.isocalendar().week
df

,date,week_of_year
0,2023-01-01,52
1,2023-01-02,1
2,2023-01-03,1
3,2023-01-04,1
4,2023-01-05,1
5,2023-01-06,1
6,2023-01-07,1
7,2023-01-08,1
8,2023-01-09,2
9,2023-01-10,2


As you can see from above, the dates are on business days, if you want to change the frequency to calendar days (known as resampling):

In [60]:
price_resampled = price.resample('D').asfreq()

In [61]:
price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


If you want to fill the na values with the most recent value, you can use the `.ffill()` method.

In [62]:
price_resampled = price.resample('D').ffill()

price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,27.724725,300.709808,111.935822,25.641571
2010-01-10,27.724725,300.709808,111.935822,25.641571
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


If you want to resample to a lower frequency (e.g. monthly) you need to provide an aggregation method:

In [63]:
price_resampled = price.resample('MS').mean()

price_resampled.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,27.166942,289.012006,110.364037,25.212407
2010-02-01,26.000370,267.179857,107.496919,23.767984
2010-03-01,29.219760,280.235245,109.849295,24.584058
2010-04-01,32.847556,278.253415,111.324726,25.646245
2010-05-01,32.888483,248.418509,109.690449,23.670255


For more resampling frequencies options, please refer to the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

> Resample price to `yearly` (start of year) frequency, use `sum` as aggregation function.

In [64]:
price_resampled=price.resample('YS').mean()
price_resampled.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,33.984624,267.544462,113.978033,22.845333
2011-01-01,47.607885,284.203368,150.379487,22.523075
2012-01-01,75.523975,321.087703,175.790281,26.485990
2013-01-01,63.212363,441.680447,176.683854,29.757183
2014-01-01,88.467678,561.342392,169.552113,40.007441


### Window functions

You can apply functions evaluated over a sliding window using the `rolling` method.

For example, to compute the 30-day moving average for Apple price:

In [65]:
price["AAPL"].rolling(30).mean()

Date
2010-01-04           NaN
2010-01-05           NaN
2010-01-06           NaN
2010-01-07           NaN
2010-01-08           NaN
                 ...    
2016-10-17    112.730333
2016-10-18    113.056000
2016-10-19    113.348000
2016-10-20    113.732666
2016-10-21    114.181667
Name: AAPL, Length: 1714, dtype: float64

By default, rolling functions require all of the values in the window to be non-NA. This behavior can be changed to account for missing data and, especially at the beginning of the time series.

In [ ]:
price["AAPL"].rolling(30, min_periods=3).mean()

> Compute a 10-day moving average for `GOOG` with a min period of 5 days.

In [66]:
price["GOOG"].rolling(10,min_periods=5).mean()


Date
2010-01-04           NaN
2010-01-05           NaN
2010-01-06           NaN
2010-01-07           NaN
2010-01-08    305.207311
                 ...    
2016-10-17    779.667004
2016-10-18    781.550006
2016-10-19    784.053009
2016-10-20    786.064007
2016-10-21    788.493005
Name: GOOG, Length: 1714, dtype: float64

## Combining and Merging Datasets

Data can be combined or merged in a number of ways:

- `merge`: connects rows in DataFrames based on one or more keys. Equivalent to database `join` operations.
- `concat`: concatenates or "stacks" together objects along an axis. Equivalent to database `union` operations.
- `combine_first`: instance method enables splicing together overlapping data to fill in missing values in one object with values from another. _We went through this in unit 7_.

### `merge`

In [67]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

In [68]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [69]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


Merging the two dataframes above constitutes a _many-to-one_ join; the data in `df1` has multiple rows labeled `a` and `b`, whereas `df2` has only one row for each value in the key column `key`.

In [ ]:
pd.merge(df1, df2)
#inner join

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If you did not specify which column(s) to join on, `merge` uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though.

In [71]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately:

In [72]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


The default merge type is `inner` join. You can specify the other options- `left, right, outer` via the `how` parameter.

In [73]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [74]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


Let's consider a _many-to-many_ join:

In [75]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                    "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], 
                    "data2": pd.Series(range(5), dtype="Int64")})

In [76]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [77]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


Since there were `three "b"` rows in the left DataFrame and `two` in the right one, there are `six "b"` rows in the result:

In [78]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


> Merge `df1` and `df2` with a left join. 

In [79]:
pd.merge(df1, df2, how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


To merge with multiple keys, pass a list of column names:

In [80]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"], 
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


If there are overlapping non-key column names:

In [81]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


You can pass `suffixes` to specify the strings to append to the overlaping names:

In [82]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


If the merge key(s) is in the index, you can pass `left_index=True` or `right_index=True` to indicate that the index should be used as the merge key.

In [83]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [84]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [85]:
right1

,group_val
a,3.5
b,7.0


In [86]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


DataFrame has a `join` method which performs a left join by default. The join key on the right dataframe has to be the index. The join key on the left dataframe can be an index or a column (by specifying the `on` parameter):

In [ ]:
left1.join(right1, on='key')

### `concat`

You can join DataFrames along any axis which is referred to as _concatenation_ or _stacking_. This is akin to database `union` operations, in any "direction" (axis).

In [87]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

Calling `concat` with these objects in a list glues together the values and indexes:

In [88]:
s1

a    0
b    1
dtype: Int64

In [89]:
s2

c    2
d    3
e    4
dtype: Int64

In [90]:
s3

f    5
g    6
dtype: Int64

By default, `concat` works along `axis="index"`, producing another Series. If you pass `axis="columns"`, the result will instead be a DataFrame:

In [91]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [92]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


The default behavior of `concat` is union (`outer` join) of the indexes, you can also intersect them by passing `join='inner'`:

In [94]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: Int64

In [98]:
# Concatenate the two Series and select the label 'a'
pd.concat([s1, s4]).loc["a"]

a    0
a    0
dtype: Int64

In [ ]:
pd.concat([s1, s4], axis="columns")

In [ ]:
pd.concat([s1, s4], axis="columns", join="inner")

When combining Series along axis="columns", pass the `keys` argument for the DataFrame column headers:

In [ ]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

> Concat `s1`, `s2` and `s3` along index and pass `keys=["one", "two", "three"]`.

For DataFrames, it will become a hierarchical index instead:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"], 
                   columns=["three", "four"])               

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

If the index does not contain any relevant data, and you want to avoid concatenating based on indexes, you can pass the `ignore_index=True` argument, this will assign a new default index:

In [ ]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), 
                   columns=["a", "b", "c", "d"])

df2 = pd.DataFrame(np.random.standard_normal((2, 3)), 
                   columns=["b", "d", "a"])

pd.concat([df1, df2], ignore_index=True)

> Concat `df1` and `df2` on the column axis but ignore the index.

## Reshaping and Pivoting Data

Reshaping or pivoting dataframes refers to the process of changing the layout of a dataframe. This is useful when you want to change the granularity of your data or when you want to convert a _wide_ dataframe into a _long_ dataframe or vice versa.

### Reshaping

In [99]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"], name="number"))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


The `stack` method pivots the columns into rows, producing a Series with a MultiIndex.

In [100]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with `unstack` , which pivots rows into columns.

By default, the innermost level is unstacked (same with stack).

In [101]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


You can unstack a different level by passing a level number or name:

In [102]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [103]:
# or just stating the name of the level
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


When you unstack in a DataFrame, the level unstacked becomes the lowest level:

In [105]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [106]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [107]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting between "Wide" and "Long" Format

Long format and wide format are two common ways of organizing data in the context of databases, spreadsheets, or data analysis. They refer to the arrangement of data rows and columns.

1. Long format

Each row typically represents a single observation or entry, and each column contains variables or attributes related to that observation. This format is also known as "tidy data" or "normalized data."

Example:

| Year | Country | Population |
| ---- | ------- | ---------- |
| 2019 | SG      | 5.7        |
| 2019 | MY      | 31.5       |
| 2019 | TH      | 69.8       |
| 2020 | SG      | 5.7        |
| 2020 | MY      | 32.7       |
| 2020 | TH      | 69.8       |

Advantages:

- It is easier to handle and analyze structured data with different attributes.
- Efficient storage for sparse data, as it avoids repeating column headers.

2. Wide format

Each row contains multiple observations or entries, and each column contains variables or attributes related to that observation.

Example:

| Year | SG   | MY   | TH   |
| ---- | ---- | ---- | ---- |
| 2019 | 5.7  | 31.5 | 69.8 |
| 2020 | 5.7  | 32.7 | 69.8 |

Advantages:

- Easier to read and understand when the number of variables is limited.
- Suitable for simple summary statistics and basic analyses.

In [108]:
price_reindex

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...,...
1709,2016-10-17,117.550003,779.960022,154.770004,57.220001
1710,2016-10-18,117.470001,795.260010,150.720001,57.660000
1711,2016-10-19,117.120003,801.500000,151.259995,57.529999
1712,2016-10-20,117.059998,796.969971,151.520004,57.250000


We can "pivot" a table from a "wide" format to a "long" format using the `melt` function.

The `date` column is the group indicator, while the other columns are data values. We need to indicate the group indicator(s):


In [109]:
melted = pd.melt(price_reindex, id_vars="Date")

melted

,Date,variable,value
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725
...,...,...,...
6851,2016-10-17,MSFT,57.220001
6852,2016-10-18,MSFT,57.660000
6853,2016-10-19,MSFT,57.529999
6854,2016-10-20,MSFT,57.250000


> Rerun `melt` and pass arguments such that the new columns are named `Company` and `Price` respectively.

In [110]:
melted = pd.melt(price_reindex, id_vars="Date", var_name="Company", value_name="Price")
melted.head()

,Date,Company,Price
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725


Using `pivot`, we can reshape back to the original layout:

In [112]:
reshaped = melted.pivot(index='Date', columns='variable', values='value')

reshaped

KeyError: 'variable'

## Data Aggregation

Data aggregation is the process of grouping data together and performing calculations on them. It is equivalent to the `GROUP BY` clause in SQL.

In [113]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None], 
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7), 
                   "data2" : np.random.standard_normal(7)})

df

,key1,key2,data1,data2
0,a,1,-1.106335,-0.645120
1,a,2,-1.196207,0.361396
2,None,1,0.812526,1.538037
3,b,2,1.356240,-0.035826
4,b,1,-0.072010,1.564644
5,a,<NA>,1.003533,-2.619745
6,None,1,0.361636,0.821903


If you want to compute the mean for each unique value in `key1`:

In [ ]:
df.groupby("key1").mean()

It does not make sense to compute the mean for `key2` since it is a categorical variable and also serves as a key.

We can select the numeric columns to compute the mean for (after the `groupby` method):

In [ ]:
df.groupby("key1")[["data1", "data2"]].mean()

Note that the following also works, since the returned result is a DataFrame, however it is less efficient as the selection/subset happens after the computation.

In [ ]:
df.groupby("key1").mean()[["data1", "data2"]]

You can group by more than 1 column. There is a useful GroupBy method `size` which returns a Series containing group sizes.

In [ ]:
df.groupby(['key1', 'key2']).size()

You can also group by other `Series`/`array`/`list` with the same length:

In [ ]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

For built-in aggregation methods in pandas, refer to the [documentation](https://pandas.pydata.org/docs/user_guide/groupby.html#built-in-aggregation-methods).

> Group by `key1` and `key2` and compute the standard deviation.

To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` method or its short alias `agg`:

In [114]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [115]:
grouped = df.groupby("key1")

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.19974,2.981141
b,1,1.42825,1.600470


You can pass a list of functions, or function names (for built-in functions) to `aggregate`: 

In [ ]:
grouped.agg([peak_to_peak, "mean", "std"])

### Apply

The most general-purpose GroupBy method is `apply`, which splits the object being manipulated into pieces, invokes the passed function on each piece, and then concatenates the pieces.

In [116]:
tips = pd.read_csv("../data/tips.csv")

tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [117]:
# add a column with the tip percentage

tips["tip_pct"] = tips["tip"] / tips["total_bill"]

Suppose we want to select the top five `tip_pct` values by group. First, write a function that selects the rows with the largest values in a particular column:

In [118]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [119]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


We can then `apply` this function by different groups using `groupby`:

In [120]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

You can pass the arguments to the function as follows:

In [121]:
tips.groupby(["smoker", "day"]).apply(top, n=2, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999

> Apply the function on `day` and `time` group.

> Create a function that selects the bottom five `tip_pct` values.
>
> Then apply it on `smoker` group.

### Transform

You can also transform your data using the `transform` method. It is similar to `apply` but imposes more restrictions on the type of function you can use. The function must:

- Produce a scalar value to be broadcast to the shape of the group chunk, or
- Return an object that is the same shape as the group chunk
- Not mutate its input

In [ ]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})

df

In [ ]:
g = df.groupby('key')['value']

g.mean()

`transform` produce a Series of the same shape as `df['value']` but with values replaced by the average grouped by `key`.

We can pass a function or function name (for built-in aggregation) to `transform`:

In [ ]:
g.transform(lambda g: g.mean())

In [ ]:
g.transform('mean')

In [ ]:
def times_two(group):
    return group * 2

g.transform(times_two)

A common transformation in data analytics / science is _standardization_ or _standard scaling_. This is where we transform the data to have a mean of 0 and a standard deviation of 1. It is also known as _z-score normalization_.

The formula for standard scaling is:

$$
z = \frac{x - \mu}{\sigma}
$$

where $x$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

We can achieve this using `transform`:

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)

or the following works too:

In [ ]:
standardized = (df['value'] - g.transform('mean')) / g.transform('std')

standardized

## Pivot Tables and Cross-Tabulation

Pivot table is a data summarization tool that is used in the context of data processing. Pivot tables are used to summarize, sort, reorganize, group, count, total or average data. It allows its users to transform columns into rows and rows into columns. It allows grouping by any data field.

In pandas, you can use the `pivot_table` method which is made possible through the `groupby` and `reshape` operations utilizing hierarchical indexing. In addition, `pivot_table` can add partial totals, also known as _margins_.

The default aggregation for `pivot_table` is mean.

In [ ]:
tips.pivot_table(index=["day", "smoker"], values=["size", "tip", "tip_pct", "total_bill"])

You can put `smoker` in the table columns and `time` and `day` in the rows:

In [ ]:
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"])

Add partial totals by passing `margins=True`:

In [ ]:
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"], margins=True)

To use other aggregation functions, pass it to the `aggfunc` keyword:

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True)

Use `fill_value` to fill missing values:

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True, fill_value=0)

> Compute the sum of `tip` in a pivot table with `day` and `time` in the rows and `smoker` in the column.

A _cross-tabulation_ or _crosstab_ is a special case of pivot table that computes group frequencies (counts):

In [ ]:
pd.crosstab(index=[tips["time"], tips["day"]], columns=tips["smoker"], margins=True)